### 붓꽃 품종 분류
- 목표: 붓꽃 3개 품종 분류
- 데이터셋: 내장 데이터
- 피쳐: 4개
- 타겟: 품종 1개
- 학습: 지도학습/분류
- 학습알고리즘: KNN

In [91]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 데이터 로드 & 전처리

In [92]:
# DF 형태로 로드
data = load_iris(as_frame=True)
print(data.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [93]:
# Bunch 인스턴스 -> Dict와 유사한 형태
featureDF = data['data']
targetSR = data['target']

In [94]:
featureDF.shape

(150, 4)

In [95]:
featureDF.head(1), 

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                5.1               3.5                1.4               0.2,)

#### 학습용, 검증용, 테스트용 데이터셋 분리

In [96]:
# 학습용 & 테스트용
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetSR,
                                                    stratify=targetSR)

In [97]:
# 학습용 & 검증용
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    stratify=y_train)

In [98]:
print(f'train DS:  {X_train.shape[0]}  {X_train.shape[0]/featureDF.shape[0]*100:.2f}%')
print(f'Val   DS:  {X_val.shape[0]}  {X_val.shape[0]/featureDF.shape[0]*100:.2f}%')
print(f'test  DS:  {X_test.shape[0]}  {X_test.shape[0]/featureDF.shape[0]*100:.2f}%')

train DS:  84  56.00%
Val   DS:  28  18.67%
test  DS:  38  25.33%


#### 교차검증 방식
- 검증 데이터셋의 

In [99]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [100]:
# 모델 인스턴스 생성
dtc_model = DecisionTreeClassifier()

##### K_fold 기반

In [101]:
# Kfold 기반-------------------------------------------

# K번째 학습 정확도 저장 리스트 선언
accuracys = []

# Kfold 인스턴스 생성	
kfold = KFold(n_splits=5)						# 기본: 5개

# K번 만큼 K개 데이터 셋으로 학습 반복
#	-> K등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스

for idx, (train_index, val_index) in enumerate(kfold.split(featureDF), 1):
    
	# print(f'train_index	:	{train_index.tolist()}')

	# X_train, X_val 데이터셋 설정
	X_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
	X_val, y_val	 = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

	# 학습진행
	dtc_model.fit(X_train, y_train)

	# 평가 (score 메서드)
	#		+) 분류:	정답(Label)에 대한 정확도 반환
	
	accuracy_train	= dtc_model.score(X_train, y_train)
	accuracy_val	= dtc_model.score(X_val, y_val)

	# 모델 점수 추가
	accuracys.append([accuracy_train, accuracy_val])

	# K번 반복 횟수 출력
	print(f'[{idx}]번째 모델 Train 정확도:	{accuracy_train}	Val 정확도:	{accuracy_val:.3f}')
	print()


[1]번째 Train 정확도:	1.0	Val 정확도:	1.000

[2]번째 Train 정확도:	1.0	Val 정확도:	0.967

[3]번째 Train 정확도:	1.0	Val 정확도:	0.833

[4]번째 Train 정확도:	1.0	Val 정확도:	0.933

[5]번째 Train 정확도:	1.0	Val 정확도:	0.800



3,4,5번째 학습 모델의 경우 => train 모델에 과적합!!

In [102]:
accuracys

[[1.0, 1.0],
 [1.0, 0.9666666666666667],
 [1.0, 0.8333333333333334],
 [1.0, 0.9333333333333333],
 [1.0, 0.8]]

In [103]:
train_score_mean = sum( [ value[0] for value in accuracys ] ) / kfold.get_n_splits()
val_score_mean	 = sum( [ value[1] for value in accuracys ] ) / kfold.n_splits

print(f'train 학습 정확도:	{train_score_mean}')
print(f'Val   학습 정확도:	{val_score_mean:.6f}')

train 학습 정확도:	1.0
Val   학습 정확도:	0.906667


In [104]:
np.mean(accuracys, axis=0)

array([1.        , 0.90666667])

##### StratifiedKFold 기반

In [105]:
# skfold 기반-------------------------------------------

# K번째 학습 정확도 저장 리스트 선언
accuracys = []

# skfold 인스턴스 생성	
skfold = StratifiedKFold(n_splits=5)						# 기본: 5개

# K번 만큼 K개 데이터 셋으로 학습 반복
#	-> K등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스

for idx, (train_index, val_index) in enumerate(skfold.split(featureDF, targetSR), 1):
    
	# print(f'train_index	:	{train_index.tolist()}')

	# X_train, X_val 데이터셋 설정
	X_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
	X_val, y_val	 = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

	# 학습진행
	dtc_model.fit(X_train, y_train)

	# 평가 (score 메서드)
	#		+) 분류:	정답(Label)에 대한 정확도 반환

	accuracy_train	= dtc_model.score(X_train, y_train)
	accuracy_val	= dtc_model.score(X_val, y_val)

	# 모델 점수 추가
	accuracys.append([accuracy_train, accuracy_val])

	# K번 반복 횟수 출력
	print(f'[{idx}]번째 모델 Train 정확도:	{accuracy_train}	Val 정확도:	{accuracy_val:.3f}')
	print()


[1]번째 Train 정확도:	1.0	Val 정확도:	0.967

[2]번째 Train 정확도:	1.0	Val 정확도:	0.967

[3]번째 Train 정확도:	1.0	Val 정확도:	0.900

[4]번째 Train 정확도:	1.0	Val 정확도:	0.967

[5]번째 Train 정확도:	1.0	Val 정확도:	1.000



In [106]:
# 모델 성능 확인
train_score_mean = sum( [ value[0] for value in accuracys ] ) / skfold.get_n_splits()
val_score_mean	 = sum( [ value[1] for value in accuracys ] ) / skfold.n_splits

print(f'train 학습 정확도:	{train_score_mean}')
print(f'Val   학습 정확도:	{val_score_mean:.6f}')

train 학습 정확도:	1.0
Val   학습 정확도:	0.960000


##### cross_val_score, corss_val_predict, cross_validate
- **교차검증 & 성능 평가 동시 진행 함수**
- 반환: **검증** 데이터셋에 대한 모델 **성능 점수 (array)**
- 장점: 자동 StratifiedKFold 실행 ==> estimator와 y가 이진or다중분류 여부 확인 

In [107]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

In [108]:
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetSR, stratify=targetSR)

In [109]:
# cross_val_predict
predict = cross_val_predict(dtc_model, X_train, y_train, cv=3)			# cv 기본: 5
print(f'predict:	{predict}')

predict:	[0 0 0 2 1 2 0 0 2 2 1 2 2 2 2 2 1 2 0 0 2 1 1 1 1 0 2 1 0 1 0 1 0 2 1 1 0
 2 1 1 1 2 1 2 1 0 0 2 0 0 0 2 2 0 1 2 2 0 2 1 2 0 0 2 0 2 0 1 0 2 2 0 2 1
 1 1 1 0 1 1 1 2 1 0 2 1 1 2 1 2 1 0 1 1 0 0 0 0 2 2 0 0 1 2 1 0 1 2 1 2 0
 0]


In [110]:
# cross_val_score 
cross_val_score(dtc_model, X_train, y_train)

array([0.86956522, 0.95652174, 0.90909091, 0.90909091, 0.95454545])

In [111]:
# cross_validate
result = cross_validate(dtc_model, X_train, y_train, 
                        cv=5,								# 폴드 수 default=5
                        return_train_score=True,			# train DS 점수까지 반환
                        return_estimator=True)				# K번째 모델의 학습 알고리즘 반환 (dtc_model 리스트 선언 가능?)
result

{'fit_time': array([0.0035398 , 0.00215912, 0.00200057, 0.00206947, 0.0019722 ]),
 'score_time': array([0.00223923, 0.00099587, 0.00239253, 0.00091791, 0.00099659]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([0.86956522, 0.95652174, 0.90909091, 0.90909091, 0.95454545]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [112]:
# test_score는 Validation_score에 해당
resultDF = pd.DataFrame(result).loc[:, ['test_score','train_score']]
resultDF

,test_score,train_score
0,0.869565,1.0
1,0.956522,1.0
2,0.909091,1.0
3,0.909091,1.0
4,0.954545,1.0


##### 최적화된 모델 추출
- (train_score - test_score) => 오름차순 정렬 => min에 해당하는 {k}번째 모델 사용!!

In [5]:
min_score = 2
bestmodel = result['estimator'][m_idx for idx in range(len(result.shape[0]) if min_score > (resultDF.loc[idx, 'train_score'] - resultDF.loc[idx, 'test_score']) m_idx=idx )]

SyntaxError: invalid syntax (2815743035.py, line 2)

In [114]:
bestmodel.predict(X_test)

array([1, 2, 0, 2, 0, 1, 2, 2, 2, 2, 1, 2, 0, 0, 2, 1, 0, 1, 0, 2, 0, 0,
       1, 1, 1, 1, 2, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 1])

In [116]:
bestmodel.score(X_test, y_test)

1.0